# Introduction to numerical algorithms
## Practice class 5

### Topics

- Eigenvalues, eigenvectors
- Applications in more physical context
- Derivatives with finite methods

In [ ]:
# Import dependencies
import subprocess
import sys

def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Package '{package}' not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"Package '{package}' installed successfully.")

import_or_install('numpy')
import numpy as np
np.random.seed(42)
import_or_install('pandas')
import pandas as pd
import_or_install('matplotlib')
import matplotlib.pyplot as plt 
import matplotlib as mpl
import_or_install('time')
import time
import_or_install('tqdm')
from tqdm import tqdm
import_or_install('scipy')
import scipy
import scipy.integrate as intgr
from matplotlib.animation import FuncAnimation, PillowWriter
from typing import Callable, Tuple

# credit to https://github.com/engineersCode/EngComp4_landlinear/tree/maste

## Task 1

Consider a system of coupled oscillators as below

![](./osc.png)

Asssume that from left to right the masses are $m_1$ and $m_2$ and the coupling strengths are $D_1$, $D_2$ and $D_3$. Let's investigate this system analytically! To start, let's try to write down the equations of motion. Simple-mindedly writing down the Newtonian equations of motions ($F=ma$) results in:

$$
m_1\ddot{x}_1 = -D_1x_1+D_2(x_2-x_1)\\
m_2\ddot{x}_2 = -D_3x_2-D_2(x_2-x_1)
$$

Now can we do anything more? Think about linear algebra!

We can rearrange the equations into matrix form, with a so-called mass ($M$) and spring matrix($D$):
$$
\textbf{M} \ddot{\underline{x}} = -\textbf{D} \underline{x}\\
\textbf{M} =  \begin{pmatrix}
        m_1 & 0 \\
        0 & m_2
    \end{pmatrix}\\
\textbf{D} =  \begin{pmatrix}
        D_1+D_2 & -D_2 \\
        -D_2 & D_2+D_3
    \end{pmatrix}\\
$$

Now if you are familiar with differential equations, this can be solved, and the solution should be something like $\ddot{x}_i=-\omega_i x_i$, hence:
$$
\underline{x}(t) = \underline{\eta}\cdot sin(\omega \cdot t + \phi)
$$
Which makes sense, even if you dont know diff. eqs, as the motion somehow should be kinda periodic, hence the sine term.

Notice the mode vector here $\underline{\eta}$. This guy will be important!

Now let's visualize the motion!

Pretending for the moment, that we know how to integrate numerically, we will let `odeint` take care of the black magic. The brute-force numerical solution is:

In [ ]:
def equations(X, t, m1, m2, k1, k2, k3):
    x1, x2, v1, v2 = X    # unpack variables
    dx1 = v1
    dx2 = v2
    dv1 = -k1/m1 * x1 + k2/m1 * (x2 - x1)
    dv2 = -k3/m2 * x2 - k2/m2 * (x2 - x1)
    dXdt = [dx1, dx2, dv1, dv2]    # pack derivatives
    return dXdt

In [ ]:
# choose parameters
m1, m2 = 1, 1
k1, k2, k3 = 1, 2, 1

# specify initial values
x1i, x2i, = 0, 0.4
v1i, v2i = 0, 0
init = [x1i, x2i, v1i, v2i]

T = 20.    # total time to solve for
time = np.arange(0, T, 0.1)    # time points to evaluate solution at

sol = intgr.odeint(equations, init, time, args=(m1, m2, k1, k2, k3))    # solve equations
X = sol[:,0:2]    # vector X consists of the first three components of the solution

In [ ]:
plt.figure()
for i in range(2):
    plt.plot(time, X[:,i], label=f'$x_{i+1}$')
plt.ylim(-1, 1)
plt.xlabel(r'$t$')
plt.ylabel(r'$x_i$')
plt.legend(ncol=3)
plt.show()

The dynamics look a bit “chaotic”. To gain more insight into the dynamics, we will decompose them into normal modes using matrix diagonalization.

Your task is:
1. Recognize that by clever rescaling you can rearrange the equation in the form of $$\ddot{\mathbf{X}} = - \mathbf{A} \cdot \mathbf{X}$$ Now you should be suspicios because this looks like a SLAE (System of Linear Algebraic Equation)! Assume $A$ can be diagonalized. Then:
\begin{equation}
\mathbf{A} = \mathbf{R} \cdot \mathbf{D} \cdot \mathbf{R}^{-1}
\end{equation}
where $\mathbf{D}$ is a diagonal matrix formed by the eigenvalues of $\mathbf{A}$, and $\mathbf{R}$ is a matrix whose columns are the corresponding eigenvectors of $\mathbf{A}$, i.e.,
\begin{equation}
\mathbf{D} = \left( \begin{array}{ccc} \lambda_1 & \\ & \lambda_2 \end{array} \right) \,, \qquad
\mathbf{R} = \left( \begin{array}{ccc} \vec{\xi^{(1)}} & \vec{\xi^{(2)}} \end{array} \right)
\end{equation}
In general case, $\mathbf{D}$ and $\mathbf{R}$ can be $N$ dimensional.
**Use `numpy` to diagonalize your matrix as such!**
2. We can use $\mathbf{R}$ to transform $\mathbf{X}$ to a new set of variables $\mathbf{X}' \equiv \mathbf{R}^{-1} \cdot \mathbf{X}$. With this transformation, the equation for $\mathbf{X}'$ simplifies to:
\begin{equation}
\ddot{\mathbf{X}'} = - \mathbf{D} \cdot \mathbf{X}'
\end{equation}
Thus, each component satisfies an equation $\ddot{x}'_i = - \lambda_i x'_i$, the solution of which is just:
\begin{equation}
x'_i(t) = A_i \sin(\omega_i t) + B_i \cos(\omega_i t)
\end{equation}
the actual value of the $A$ and $B$ are depending on the initial condition as,
$$
A_i = \dot{X}_i'(0) / \omega_i  = \{\mathbf{R}^{-1} \cdot \mathbf{X}(0)\}_i/\omega_i \\
B_i = X_i'(0) = \{\mathbf{R}^{-1} \cdot \mathbf{X}(0)\}_i
$$
 and $\omega_i = \sqrt{\lambda_i}$ (we need $\lambda_i \geq 0$, which is true for $\mathbf{A}$, as shown below). Therefore, the solution for the original variables can be written as a sum over **normal modes**:
\begin{equation}
\mathbf{X}(t) = \mathbf{R} \cdot \mathbf{X}'(t) = \sum_i \vec{\xi^{(i)}} (A_i \sin(\omega_i t) + B_i \cos(\omega_i t)
\end{equation}
Each mode has its own frequency of oscillation $\omega_i$, and the normalized eigenvector $\vec{\xi^{(i)}}$ represents the "direction" of each mode, whereas $A_i$ and $B_i$ represents the "amplitude" of each mode.
**Find the eigenvectors and eigenvalues (mode vectors and eigenfrequencies in fact)**
3. Substitute back to the solution! Plot the trajectories for each mass found by the integration and found via eigenvalue problem!
4. Decompose the motion of the different modes and plot them!
5. Knowing the modes of the system, check the trajectories for different initial conditions and plot them! Notice that in the last equations the initial conditions matter and appear in the solutions. Where exactly do you have to use the initial conditions, when computing the trajectories with the help of the modes?

If you have the modes, then with this function you can animate:

In [ ]:
import matplotlib.animation as anim

plt.rcParams["animation.html"] = "jshtml"
fig, ax = plt.subplots(figsize=(4,4))
ax.set_xlim(-1, 3)
ax.set_ylim(-0.5, 2.5)
ax.axis('off')
ax.text(-1, 0.2, 'all modes')
ax.text(-1, 1.2, 'mode 1')
ax.text(-1, 2.2, 'mode 2')
p1, = ax.plot([], [], 'o-')    # plot masses in mode 1
p2, = ax.plot([], [], 'o-')    # plot masses in mode 2
p0, = ax.plot([], [], 'o-')    # plot masses in all modes
offset = np.arange(2)    # add natural length of springs

def animate(t):
    p1.set_data(R[:,0]*modes[0,t] + offset, [1, 1])
    p2.set_data(R[:,1]*modes[1,t] + offset, [2, 2])
    p0.set_data(X_sum[:,t] + offset, [0, 0])

mov = anim.FuncAnimation(fig, animate, frames=len(time), interval=50)
plt.close()

In [ ]:
mov

Finally, we can plot the configuration of the system in a 3-D space to visualize the superposition of the modes.

In [ ]:
fig2D = plt.figure(figsize=(5, 5))
ax = fig2D.add_subplot()
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_xlabel(f'$x_1$')
ax.set_ylabel(f'$x_2$')
ax.plot([R[0, 0], -R[0, 0]], [R[1, 0], -R[1, 0]], 'blue')    # plot eigenvector 1
ax.plot([R[0, 1], -R[0, 1]], [R[1, 1], -R[1, 1]], 'orange')  # plot eigenvector 2
p1, = ax.plot([], [], 'o', label='Mode 1')    # plot displacements in mode 1
p2, = ax.plot([], [], 'o', label='Mode 2')    # plot displacements in mode 2
p0, = ax.plot([], [], '-', label='Total Displacement')    # plot total displacements

def animate2D(t):
    p1.set_data([R[0, 0] * modes[0, t]], [R[1, 0] * modes[0, t]])
    p2.set_data([R[0, 1] * modes[1, t]], [R[1, 1] * modes[1, t]])
    p0.set_data(X_sum[0, :t + 1], X_sum[1, :t + 1])
    ax.legend()

mov2D = anim.FuncAnimation(fig2D, animate2D, frames=len(time), interval=50)
plt.close()

In [ ]:
mov2D

## Warmup

Use vectorized methods to calculate the following quantities:
1. $y_i = x_{i+1} - x_{i}$
2. $y_i = x_{i} - x_{i-2}$
3. $y_i = 3x_{i}-4x_{i-1}+x_{i-2}$

Be careful with the length of the resultant array!
Use the provided array as input!

In [ ]:
t=np.linspace(0,np.pi/2,10)
x=np.sin(t)

### Task 2 - Compare finite difference methods

In this task you are going to compare the different finite difference derivative methods. In order to do that you are going to write a function what calculates finite differences.

**Central**
| Order | Accuracy | -2   | -1   | 0    | 1   | 2     |
|-------|----------|------:|------:|------:|-----:|-------:|
| 1     | 2        |      | -1/2 | 0    | 1/2 |       |
| 1     | 4        | 1/12 | −2/3 |   0  | 2/3 | −1/12 |

**Forward**
| Order | Accuracy  | 0    | 1   | 2     |
|-------|-----------|------:|-----:|-------:|
| 1     | 1         |  −1  |  1  |       |
| 1     | 2         | −3/2 |  2  |  −1/2 |

**Backward**
| Order | Accuracy | -2   | -1   | 0    |
|-------|----------|------:|------:|------:|
| 1     | 1        |      |  −1  |  1   |
| 1     | 2        |  1/2 |  −2  | 3/2  |

1. Write a function ` my_der_calc(f,a,b,N,option)`, with the output as `[df,X]`, where `f` is a function object, `a` and `b` are scalars such that `a < b`, `N` is an integer bigger than 10, and `option` is the string forward, backward, or central. Let `x` be an array starting at `a`, ending at `b`, containing `N` evenly spaced elements, and let $y$ be the array $f(x)$. The output argument, `df`, should be the numerical derivatives computed for $x$ and $y$ according to the method defined by the input argument, option. The output argument `X` should be an array the same size as `df` containing the points in `x` for which `df` is valid. Specifically, the forward difference method “loses” the last point, the backward difference method loses the first point, and the central difference method loses the first and last points.

2. Add an other argument to your previous funtion, `accuracy`, what is an integer and can be 1 or 2 based on the rows of the tables above. (And even larger values, see: https://en.wikipedia.org/wiki/Finite_difference_coefficient). 

3. Test your function. Use the code below to test you function on two different predefined use cases. 



In [ ]:
# Test code for Task 2.
x = np.linspace(0, 2*np.pi, 100)
f = lambda x: np.sin(x)
accuracy=1
[dyf, Xf] = my_der_calc(f, 0., 2*np.pi, 10, 'forward',accuracy)
[dyb, Xb] = my_der_calc(f, 0., 2*np.pi, 10, 'backward',accuracy)
[dyc, Xc] = my_der_calc(f, 0., 2*np.pi, 10, 'central',accuracy)
plt.figure(figsize = (12, 8))
plt.plot(x, np.cos(x), label = 'analytic')
plt.plot(Xf, dyf, label = 'forward')
plt.plot(Xb, dyb, label = 'backward')
plt.plot(Xc, dyc, label = 'central')
plt.legend()
plt.title('Analytic and Numerical Derivatives of Sine')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
#Test code for task 2
x = np.linspace(0, np.pi, 1000)
f = lambda x: np.sin(np.exp(x))
accuracy=1
[dy10, X10] = my_der_calc(f, 0., np.pi, 10, 'central',accuracy)
[dy20, X20] = my_der_calc(f, 0., np.pi, 20, 'central',accuracy)
[dy100, X100] = my_der_calc(f, 0., np.pi, 100, 'central',accuracy)
plt.figure(figsize = (12, 8))
plt.plot(x, np.cos(np.exp(x))*np.exp(x), label = 'analytic')
plt.plot(X10, dy10, label = '10 points')
plt.plot(X20, dy20, label = '20 points')
plt.plot(X100, dy100, label = '100 points')
plt.legend()
plt.title('Analytic and Numerical Derivatives of Sine')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### **Task 3 — Convergence and Roundoff Error Study**

1. **Goal:**
   Examine how the finite difference error changes as the step size ( h ) decreases.

2. **Steps:**

   * Use your `my_der_calc` function with the test function $ f(x) = \sin(x) $.
   * Its exact derivative is $ f'(x) = \cos(x) $.
   * Compute the derivative numerically over ([0, \pi]) using the **central difference** method with **2nd-order accuracy**.
   * Vary the number of points `N` (e.g. `[10, 20, 40, 80, 160, 320, 640, 1280, 2560]`).
   * For each `N`, compute the **maximum absolute error**:
     $$
     E(N) = \max | f'(x_i) - f'_{\text{num}}(x_i) |
     $$
   * Plot $ E(N) $ versus $ h = \frac{b-a}{N-1} $ on a log-log scale.

3. **Expected behavior:**

   * Initially, as $ h $ decreases, the error $ E $ decreases approximately as $O(h^p)$, where $ p $ is the order of the method.
   * But for very small $ h $, floating-point **round-off error** becomes dominant, and the error **stops decreasing** (or even increases).
   * This demonstrates the balance between **truncation error** and **round-off error** in numerical differentiation.

4. **Optional extension:**

   * Estimate the **experimental order of convergence (EOC)** by comparing consecutive error values:
     $$
     p \approx \frac{\log(E_i / E_{i+1})}{\log(h_i / h_{i+1})}
     $$

**Remark:** use first order derivation scheme for this task.


# Homework 1

In this task we will compare the efficiency of different inverse matrix calculation methods. Make sure you have `time` and `memory_usage` or `tracemalloc` packages installed! The methods:
1. np.linalg.inv 
2. LU decomposition - you can do this always
3. QR decomposition - only if you have a  matrix with linearly independent columns
4. Cholesky decomposition - only for a positive definite matrix
5. Gauss-Jordan method - always, if you have the conditions for GE
6. Montante method

Check the memory usage and time for each method for different matrix sizes. Start at $n=100$ matrix size and try to go up at least to $n=2500$ or above. Summarize the results on plots

# Homework 2

Below you have a transformation corresponding to the matrix

$$C = \begin{bmatrix} 2 & 1 \\ 1 & 2 \end{bmatrix}$$

1. Plot the natural basis and then the transformed natural basis after applying the above matrix.
2. Take min. 41 vectors in the natural basis that are spanning starting from the origin of the natural basis and ending along a unit circle. Plot this.
3. Apply the above transformation on the vectors and visualize the result! If you did everything correctly, then the shape you will see is an ellipsis. What will the ellipsis correspond to for a human being living in basis C?
4. Write a program that calculates the norm of a vector in a given basis

For plotting the vectors you can use `plot_vector` from `plot_helper` or write your own function!

## Homework 3

Repeat the same as in **Task 4** but for the following system:

![](https://physicscourses.colorado.edu/phys3210/phys3210_sp20/images/fig31-spring-ring.png)

I would suggest to keep the masses and couplings as variables, so your code works for non identical case as well, just as in the problem we solved in the class.